# 4. Append 2024

## 4a. List, virtualize and concatenize datasets

2023-09-04 to 2023-12-31 chunking:

* `sea_ice_fraction` and `mask` from 2023-09-04 to the end of 2023 have chunking (1, 4500, 9000) (previously (1, 1447, 2895))
* `analysed_sst` and `analysis_error` from 2023-09-04 to the end of 2023 have chunking (1, 3600, 7200) (previously (1, 1023, 2047))

2024 to date chunking:
* `sea_ice_fraction` and `mask` start 2024 with chunking (1, 4500, 9000) and then changes on 2024-03-24 to (1, 1023, 2047) - **never seen this chunk shape for these variables before**.
* `analysed_sst` and `analysis_error` start 2024 with chunking (1, 3600, 7200) and then changes on 2024-03-24 to (1, 1023, 2047) (original chunking)

It would be nice to just append to the existing store, using zarr for 2023-09-04 to sometime in 2024 when it changes, but then we couldn't go back to virtual references since `sea_ice_fraction` and `mask` are using a chunk shape never used before. A combination of zarr + virtual references seems possible but messy. Plus we still have to write over 6 months as zarr.



In [81]:
dates = ["2024-01-01", "2024-12-31"]
mur_sst_files_2024 = helpers.list_mur_sst_files(start_date=dates[0], end_date=dates[1])
mur_sst_dmrpps_2024 = [f + '.dmrpp' for f in mur_sst_files_2024]

In [82]:
# sanity check
len(mur_sst_dmrpps_2024)

366

In [83]:
vdss = [helpers.open_virtual(f) for f in mur_sst_dmrpps_2024]

In [86]:
first_chunk_shape = vdss[0]['analysed_sst'].chunks
for vds in vdss:
    if vds['analysed_sst'].chunks != first_chunk_shape:
        print(vds.analysed_sst.data.manifest.dict()['0.0.0']['path'])
        break

s3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20240324090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc


In [28]:
# create the virtual store
virtual_ds_2024 = helpers.create_virtual_ds(dmrpps=mur_sst_dmrpps_2024)

2025-01-24 17:14:35,865 - distributed.worker - ERROR - Compute Failed
Key:       ('from_sequence-open_virtual-reduce_via_concat-part-d183d7f52955da84300603ccd0058afc', 76)
State:     executing
Task:  <Task ('from_sequence-open_virtual-reduce_via_concat-part-d183d7f52955da84300603ccd0058afc', 76) _execute_subgraph(...)>
Exception: "ValueError('Cannot concatenate arrays with inconsistent chunk shapes: (1, 1023, 2047) vs (1, 4500, 9000) .Requires ZEP003 (Variable-length Chunks).')"
Traceback: '  File "/opt/conda/lib/python3.11/site-packages/dask/bag/core.py", line 2504, in empty_safe_apply\n    return func(part)\n           ^^^^^^^^^^\n  File "/home/jovyan/icechunk-nasa/icechunk-nasa/notebooks/mur-sst/helpers.py", line 33, in reduce_via_concat\n    return xr.concat(\n           ^^^^^^^^^^\n  File "/opt/conda/lib/python3.11/site-packages/xarray/core/concat.py", line 277, in concat\n    return _dataset_concat(\n           ^^^^^^^^^^^^^^^^\n  File "/opt/conda/lib/python3.11/site-packages/xar

ValueError: Cannot concatenate arrays with inconsistent chunk shapes: (1, 1023, 2047) vs (1, 4500, 9000) .Requires ZEP003 (Variable-length Chunks).

## 3b. Write to icechunk

In [ ]:
%%time
session = repo.writable_session(branch="main")
store = session.store
virtual_ds_2024.virtualize.to_icechunk(store)
session.commit("Wrote 2024 data")

## 3c. Validate

In [ ]:
%%time
helpers.validate_data(store, dates=dates, fs=fs)